In [ ]:
from sklearn.model_selection import train_test_split
# from autogluon.tabular import TabularPredictor
from meteostat import Point, Daily
from multiprocessing import  Pool
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import warnings

#Constants
ALTITUDE = 10

In [2]:
df_cargo = pd.read_csv('db-cargo-delays-2016-deutschland.csv')
df_cargo.head()

,BST_NR_8,Land,PROD_DATUM,Zugfahrten,Verspaetungsminuten,dayofweek,weekofyear,month
0,80197079,DEUTSCHLAND,2016-03-16,39,49,3,11,3
1,80160168,DEUTSCHLAND,2016-01-11,12,0,1,2,1
2,80105122,DEUTSCHLAND,2016-04-30,16,2,6,17,4
3,80190793,DEUTSCHLAND,2016-07-31,14,0,7,30,7
4,80231464,DEUTSCHLAND,2016-04-05,10,2,2,14,4


In [12]:
df_geo = pd.read_csv('geo-bahnstellen-export.csv', delimiter=';')
df_geo = df_geo.rename(columns={'BST8': 'BST_NR_8', 'LAT': 'latitude', 'LON': 'longitude'})
df_geo.head()

,BST_NR_8,BST_NAME,LAND,latitude,longitude
0,11759,NaN,NaN,NaN,NaN
1,28100,SWIEBODZIN,NaN,52.247296,15.533572
2,254581,NaN,NaN,NaN,NaN
3,824532,NaN,NaN,NaN,NaN
4,999995,NaN,NaN,NaN,NaN


In [13]:
len(df_cargo), len(df_geo)

(771621, 51432)

In [15]:
df = df_cargo.merge(df_geo[['BST_NR_8', 'latitude', 'longitude']])
df['PROD_DATUM'] = pd.to_datetime(df['PROD_DATUM'])
df = df.reset_index()
del df_cargo
del df_geo

df.head()

,BST_NR_8,Land,PROD_DATUM,Zugfahrten,Verspaetungsminuten,dayofweek,weekofyear,month,latitude,longitude
0,80197079,DEUTSCHLAND,2016-03-16,39,49,3,11,3,49.42045,7.613668
1,80197079,DEUTSCHLAND,2016-10-11,38,74,2,41,10,49.42045,7.613668
2,80197079,DEUTSCHLAND,2016-08-29,27,33,1,35,8,49.42045,7.613668
3,80197079,DEUTSCHLAND,2016-02-25,53,85,4,8,2,49.42045,7.613668
4,80197079,DEUTSCHLAND,2016-05-28,39,38,6,21,5,49.42045,7.613668


In [8]:
df = df.drop(['index'], axis=1)
df = df.reset_index(drop=True)
df.to_csv('df-merged.csv', index=False)

In [59]:
df['year'] = df['PROD_DATUM'].dt.year
df['day'] = df['PROD_DATUM'].dt.day

df.head()

,BST_NR_8,Land,PROD_DATUM,Zugfahrten,Verspaetungsminuten,dayofweek,weekofyear,month,latitude,longitude,year,day
0,80197079,DEUTSCHLAND,2016-03-16,39,49,3,11,3,49.42045,7.613668,2016,16
1,80197079,DEUTSCHLAND,2016-10-11,38,74,2,41,10,49.42045,7.613668,2016,11
2,80197079,DEUTSCHLAND,2016-08-29,27,33,1,35,8,49.42045,7.613668,2016,29
3,80197079,DEUTSCHLAND,2016-02-25,53,85,4,8,2,49.42045,7.613668,2016,25
4,80197079,DEUTSCHLAND,2016-05-28,39,38,6,21,5,49.42045,7.613668,2016,28


In [16]:
def get_weather_data(y, m, d, lat, lon):
    # Set time period
    start = datetime(y, m, d)
    end = datetime(y, m, d)

    # Create Point for Vancouver, BC
    vancouver = Point(lat, lon, ALTITUDE)

    # Get daily data for 2018
    weather_data = Daily(vancouver, start, end)
    weather_data = weather_data.fetch()
    return weather_data

In [17]:
def create_weather_data(dataset): 
    print("Start Time: %s" % time.ctime()) # Start Time: Fri Jan  7 16:53:54 2022
    df_result = get_weather_data(dataset.loc[0]['year'], 
                                 dataset.loc[0]['month'],
                                 dataset.loc[0]['day'],
                                 dataset.loc[0]['latitude'],
                                 dataset.loc[0]['longitude'],
                                )
    df_result['df_index'] = 0

    for idx, row in dataset.iterrows():
        print('row index:', idx)
        weather_data = get_weather_data(row['year'], 
                                        row['month'],
                                        row['day'],
                                        row['latitude'],
                                        row['longitude'])
        weather_data['df_index'] = idx
        df_result = pd.concat([df_result, weather_data])

    print("End Time: %s" % time.ctime())
    # Drop the first row
    df_result = df_result.iloc[1: , :]
    # Reset indices
    df_result = df_result.reset_index()
    return df_result



In [9]:
df = pd.read_csv('df-merged.csv')
df.head()

,BST_NR_8,Land,PROD_DATUM,Zugfahrten,Verspaetungsminuten,dayofweek,weekofyear,month,latitude,longitude,year,day
0,80197079,DEUTSCHLAND,2016-03-16,39,49,3,11,3,49.42045,7.613668,2016,16
1,80197079,DEUTSCHLAND,2016-10-11,38,74,2,41,10,49.42045,7.613668,2016,11
2,80197079,DEUTSCHLAND,2016-08-29,27,33,1,35,8,49.42045,7.613668,2016,29
3,80197079,DEUTSCHLAND,2016-02-25,53,85,4,8,2,49.42045,7.613668,2016,25
4,80197079,DEUTSCHLAND,2016-05-28,39,38,6,21,5,49.42045,7.613668,2016,28


In [19]:
def parallelize_dataframe(dataset, func, n_cores=6):
    df_split = np.array_split(dataset, n_cores)
    pool = Pool(n_cores)
    create_weather = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return create_weather

In [11]:
# np.array_split(df, 6)

In [ ]:
create_weather = parallelize_dataframe(df.iloc[:600 , :], create_weather_data)
create_weather